# An extensive comparison and visualisation of popular optimisation alogrithms for gradient descent

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation


In [2]:
# example function (quadratic)
"""

def function(x):
    return x**2 + 5*x + 3

def function_grad(x):
    return 2*x + 5

other functions for testing:

(cubic)
def function(x):
    return x**3 - 3*x**2 + 2

def function_grad(x):
    return 3*x**2 - 6*x

"""

def function(x):
    return np.sin(3 * x) + 0.5 * x**2

def function_grad(x):
    return 3 * np.cos(3 * x) + x



In [32]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D

def create_3d_gif(history, filename):
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')

    # Define the x and y range for the function
    x_min, x_max = min(history) - 2, max(history) + 2
    x_vals = np.linspace(x_min, x_max, 100)
    y_vals = np.linspace(x_min, x_max, 100)
    X, Y = np.meshgrid(x_vals, y_vals)

    # Define the function (assuming f(x, y) = f(x) + f(y))
    Z = function(X) + function(Y)

    # Scatter plot of optimization path
    history_x = np.array(history)
    history_y = np.array(history)  # Assuming single-variable optimization (x = y)
    history_z = function(history_x) + function(history_y)

    def update(i):
        ax.clear()
        ax.plot_surface(X, Y, Z, cmap='viridis', alpha=0.8)
        ax.scatter(history_x[:i], history_y[:i], history_z[:i], color='red', s=40)

        ax.set_xlabel("x")
        ax.set_ylabel("y")
        ax.set_zlabel("f(x, y)")
        ax.set_title(f"Optimizer Path - Iteration {i+1}")

    ani = animation.FuncAnimation(fig, update, frames=len(history), repeat=False)
    ani.save(filename, writer="pillow", fps=10)
    plt.close(fig)

    print(f"3D GIF saved as {filename}")


a function to create a gif of the training process



In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

def create_gif(history, filename):
    fig, ax = plt.subplots(figsize=(8, 5))

    x_min, x_max = min(history) - 10, max(history) + 10
    x_vals = np.linspace(x_min, x_max, 500)
    y_vals = function(x_vals)

    y_min, y_max = np.min(y_vals), np.max(y_vals)

    y_margin = (y_max - y_min) * 0.2  # 20% margin
    y_min -= y_margin
    y_max += y_margin

    def update(i):
        ax.clear()
        ax.plot(x_vals, y_vals, label="Function f(x)", color="blue", linewidth=2)
        ax.scatter(history[i], function(history[i]), color="red", s=100, label="Current x")

        ax.set_xlim(x_min, x_max)
        ax.set_ylim(y_min, y_max)

        ax.set_xlabel("x")
        ax.set_ylabel("f(x)")
        ax.set_title(f"Gradient Descent Progress (Iteration {i+1})")
        ax.legend()
        ax.grid(True)

    ani = animation.FuncAnimation(fig, update, frames=len(history), repeat=False)
    ani.save(filename, writer="pillow", fps=10)
    plt.close(fig)

    print(f"GIF saved as {filename}")


## Gradient Descent

  Let θ be the parameter vector and J(θ) be the objective function

  - Update rule: θₜ₊₁ = θₜ - η∇J(θₜ)

  - ∇J(θₜ) is the gradient of the objective function with respect to θ

In [39]:
def gradient_descent(lr, initial_x, iters, eps = 1e-4):

  """
    Parameters:
    - lr: learning rate, which controls step size in each iteration
    - initial_x: starting point for gradient descent
    - iters: maximum number of iterations to run
    - eps: convergence threshold. Stops if step size is smaller than this value

    Returns:
    - x: approximate local minimum found
    - history: list of x values at each iteration

  """

# initialising x at any given point (in practice, we want to randomise this)
  x = initial_x
  history = []

  for i in range(iters):
    gradient = function_grad(x)   # calcs gradient of a given point
    x_new = x - lr * gradient     # x_new = x_old - n(∇f(x))
    history.append(x_new)         # to store values of x for visualisation later

    # checking for convergence
    if abs(x_new - x) < eps:
        print(f"Converged after {i+1} iterations.")
        print(f"Iteration {i+1}: x = {x_new}, f(x) = {function(x_new)}")
        return x_new, history     # returns results early if converged

    x = x_new   # for next iter we update x_new -> x_old

  # if the loop completes without convergence, returns the last computed x
  print(f"Iteration {i+1}: x = {x}, f(x) = {function(x)}")

  return x, history


In [5]:
x, history = gradient_descent(0.1, 5, 200)

Converged after 31 iterations.
Iteration 31: x = 1.4079621743825572, f(x) = 0.10814209088122684


In [6]:
filename = "gradient_descent.gif"

create_gif(history, filename)
print("GIF saved as 'gradient_descent.gif'")

GIF saved as gradient_descent.gif
GIF saved as 'gradient_descent.gif'


In [34]:
create_3d_gif(history, "gradient_descent_3d.gif")


3D GIF saved as gradient_descent_3d.gif


## Stochastic Gradient Descent

It approximates the gradient using a small subset of data (or a single sample) in each iteration.

  - Same update rule as standard gradient descent, but ∇Jᵢ(θₜ) is the gradient computed on a random sample i or a mini-batch.

  - This introduces noise but allows for faster iterations and can help escape local minima.


In [35]:
def stochasticGD(lr, initial_x, iters, eps = 1e-4):

  """

  Parameters: (same as previous)
    - lr: learning rate
    - initial_x: starting point for gradient descent
    - iters: maximum number of iterations
    - eps: convergence threshold

    Returns:
    - x: approximate local minimum found
    - history: list of x values at each iteration

  """

  x = initial_x
  history = []

  for i in range(iters):
      # to introduce randomness (simulate noisy gradients)
      noise = np.random.normal(0, 0.1)  # small gaussian noise
      gradient = function_grad(x) + noise

      x_new = x - lr * gradient
      history.append(x_new)

      # check for convergence
      if abs(x_new - x) < eps:
          print(f"Converged after {i+1} iterations.")
          print(f"Iteration {i+1}: x = {x_new}, f(x) = {function(x_new)}")
          return x_new, history

      x = x_new

  print(f"Iteration {iters}: x = {x}, f(x) = {function(x)}")

  return x, history

In [36]:
sgd_x, sgd_history = stochasticGD(lr = 0.1, initial_x = 5, iters = 200)

Iteration 200: x = -0.920674240127139, f(x) = -1.54038866584046


In [37]:
filename = "sgd.gif"

create_gif(sgd_history, filename)
print("GIF saved as 'sgd.gif'")

GIF saved as sgd.gif
GIF saved as 'sgd.gif'


In [38]:
create_3d_gif(sgd_history, "sgd_3d.gif")


3D GIF saved as sgd_3d.gif


## Batch Gradient Descent


In [10]:
def batch_gradient_descent(lr, initial_x, iters, batch_size, eps = 1e-4):

    """

    Parameters (extra):
        - batch_size: number of random samples used to approximate the gradient

    """

    x = initial_x
    history = [x]

    # computes batch gradient approximation by averaging gradients over batch_size samples
    for i in range(iters):
        batch_grad = np.mean([function_grad(x + np.random.uniform(-0.5, 0.5)) for _ in range(batch_size)])

        # updates x based on the batch gradient and learning rate
        x_new = x - lr * batch_grad

        history.append(x_new)

        if abs(x_new - x) < eps:
            print(f"Converged after {i+1} iterations.")
            print(f"Final: x = {x_new}, f(x) = {function(x_new)}")
            return x_new, history

        x = x_new

    print(f"Max iterations reached ({iters}).")
    print(f"Final: x = {x}, f(x) = {function(x)}")
    return x, history



In [11]:
batch_x, batch_history = batch_gradient_descent(lr = 0.02, initial_x = 5, iters = 201, batch_size = 5)

Max iterations reached (201).
Final: x = 1.3019475376985266, f(x) = 0.15553795762244038


In [12]:
filename = "batch_gd.gif"

create_gif(batch_history, filename)
print("GIF saved as 'batch_gd.gif'")

GIF saved as batch_gd.gif
GIF saved as 'batch_gd.gif'


## SGD with Momentum

- Speeds up SGD by adding momentum in the relevant direction to overcome saddle points and local minima.

- The idea is to add a fraction of the previous update to the current update, helping to accelerate convergence and reduce oscillation

Formula:

- It introduce a velocity vector vₜ = βvₜ₋₁ + η∇J(θₜ)

- Update rule: θₜ₊₁ = θₜ - ηvₜ

- Where β is the momentum coefficient (typically 0.9)




In [13]:
def function(x):
    return x**2 + 3*x + 5

def function_grad(x):
    return 2*x + 3

In [14]:
def sgd_with_momentum(lr, initial_x, iters, beta = 0.9, eps = 1e-4):

    """

    Parameters:
        - beta: Momentum coefficient (default 0.9)

    """

    x = initial_x
    v = 0  # initialize momentum term
    history = [x]

    for i in range(iters):
        gradient = function_grad(x)
        v = beta * v + lr * gradient  # update velocity term
        x_new = x - lr * v

        history.append(x_new)

        # Convergence check
        if abs(x_new - x) < eps:
            print(f"Converged after {i+1} iterations.")
            print(f"Final: x = {x_new}, f(x) = {function(x_new)}")
            return x_new, history

        x = x_new  # Update x for next iteration

    print(f"Max iterations reached ({iters}).")
    print(f"Final: x = {x}, f(x) = {function(x)}")
    return x, history


In [15]:
x_opt, sgd_mom_history = sgd_with_momentum(lr = 0.1, initial_x = -4, iters = 100)

Converged after 93 iterations.
Final: x = -1.5187757644204394, f(x) = 2.750352529329572


In [16]:
filename = "sgd_momentum.gif"
create_gif(sgd_mom_history, filename)

GIF saved as sgd_momentum.gif


# AdaGrad

- Adds to SGD by introducing learning rate as a parameter, gets modified according to previous gradients for each parameter

- If the previous gradient is large, the learning rate gets smaller. However this causes a problem in later step as th elearning rate becomes too small as gradient keeps accumulating.

- Useful for sparse data

In [17]:
def function(x):
    return x**2 + 3*np.sin(x)

def function_grad(x):
    return 2*x + 3*np.cos(x)  # Derivative of f(x)


In [18]:
def AdaGrad(lr, initial_x, iters, eps = 1e-4):
  g = 0    # running sum of squared gradients (lr)
  x = initial_x
  history = [x]
  new_lr = 0

  for i in range(iters):
    gradient = function_grad(x)
    g += gradient ** 2

    new_lr = lr / (np.sqrt(g) + 1e-8)

    x -= new_lr * gradient

    history.append(x)

    # Convergence check
    if abs(gradient) < eps:
        print(f"Converged after {i+1} iterations.")
        print(f"Final: x = {x}, f(x) = {function(x)}")
        return x, history

  print(f"Max iterations reached ({iters}).")
  print(f"Final: x = {x}, f(x) = {function(x)}")
  return x, history


In [19]:
x_adagrad, adagrad_history = AdaGrad(lr = 0.9, initial_x = 4, iters = 100)

Converged after 42 iterations.
Final: x = -0.9148453218403361, f(x) = -1.5404628054370935


In [20]:
adagrad_history

[4,
 3.100000001490296,
 2.6783416864882197,
 2.350592220333358,
 2.0509205542565296,
 1.7544007708293532,
 1.450173540477058,
 1.1361543896087731,
 0.8177474984277625,
 0.5062208874883785,
 0.21546880770747873,
 -0.04208298667488036,
 -0.258718338567969,
 -0.4325694880228168,
 -0.566660923554137,
 -0.6668815510744233,
 -0.7400239977779683,
 -0.7924821055783032,
 -0.8296390263365665,
 -0.8557273760429119,
 -0.8739320527041523,
 -0.8865812475141349,
 -0.8953443273471251,
 -0.901402780628205,
 -0.9055854471745448,
 -0.908470286768775,
 -0.9104586629924687,
 -0.911828518094478,
 -0.9127719540149515,
 -0.913421567402206,
 -0.9138687984804151,
 -0.9141766659924723,
 -0.9143885824524957,
 -0.9145344451167865,
 -0.914634839376132,
 -0.9147039370602219,
 -0.9147514936969965,
 -0.9147842242979077,
 -0.9148067507876938,
 -0.9148222543248486,
 -0.9148329243760266,
 -0.9148402678428031,
 -0.9148453218403361]

In [21]:
# Parameters

filename = "adagrad.gif"
create_gif(adagrad_history, filename)

GIF saved as adagrad.gif


## Adagrad experiments

In [22]:
def AdaGrad(lr, initial_x, iters, eps = 1e-5):
  g = 0    # running sum of squared gradients (lr)
  x = initial_x
  history = [x]
  new_lr = 0

  for i in range(iters):
    gradient = function_grad(x)
    g += gradient ** 2

    new_lr = lr / (np.sqrt(g) + 1e-1)

    x -= new_lr * gradient

    history.append(x)

    # Convergence check
    if abs(gradient) < eps:
        print(f"Converged after {i+1} iterations.")
        print(f"Final: x = {x}, f(x) = {function(x)}")
        return x, history

  print(f"Max iterations reached ({iters}).")
  print(f"Final: x = {x}, f(x) = {function(x)}")
  return x, history


In [26]:
x_adagrad_exp, adagrad_exp_history = AdaGrad(lr = 0.3, initial_x = 4, iters = 200)

Max iterations reached (200).
Final: x = -0.9121287137042742, f(x) = -1.5404465263496046


In [24]:
adagrad_history[:-1:5]

[4,
 1.7544007708293532,
 0.21546880770747873,
 -0.6668815510744233,
 -0.8739320527041523,
 -0.908470286768775,
 -0.9138687984804151,
 -0.9147039370602219,
 -0.9148329243760266]

In [25]:
# Parameters

filename = "adagrad_exp.gif"
create_gif(adagrad_exp_history, filename)

GIF saved as adagrad_exp.gif


In [33]:
create_3d_gif(adagrad_exp_history, "optimizer_path_3d.gif")


3D GIF saved as optimizer_path_3d.gif


# RMSProp

In [27]:
def RMSProp(lr, initial_x, iters, eps = 1e-5, beta = 0.9):     # new term - beta introduced
  g = 0    # running sum of squared gradients (lr)
  x = initial_x
  history = [x]
  new_lr = 0

  for i in range(iters):
    gradient = function_grad(x)
    # uppdated moving average of squared gradients
    g = beta * g +(1 - beta) * gradient ** 2

    new_lr = lr / (np.sqrt(g) + 1e-5) # adaptive learning rate

    x -= new_lr * gradient

    history.append(x)

    # convergence check
    if abs(gradient) < eps:
        print(f"Converged after {i+1} iterations.")
        print(f"Final: x = {x}, f(x) = {function(x)}")
        return x, history

  print(f"Max iterations reached ({iters}).")
  print(f"Final: x = {x}, f(x) = {function(x)}")
  return x, history


In [28]:
x_rmsprop, rmsprop_history = AdaGrad(lr = 0.01, initial_x = 4, iters = 200)

Max iterations reached (200).
Final: x = 3.741470279797083, f(x) = 12.304975445450939


In [31]:
# Parameters

filename = "rmsprop.gif"
create_gif(rmsprop_history, filename)

GIF saved as rmsprop.gif


# Adam Optimiser